## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 10000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    },    
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print("You are using TensorFlow version", tf .__version__)

You are using TensorFlow version 1.15.2


## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
Cloning into 'tensorflow-object-detection-faster-rcnn'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 24.83 MiB | 16.77 MiB/s, done.
Resolving deltas: 100% (428/428), done.
/content/tensorflow-object-detection-faster-rcnn
Already up to date.


## Install required packages

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
     |████████████████████████████████| 358kB 8.5MB/s 
Selecting previously unselected package python-bs4.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpa

**Data Preparation**

In [ ]:
#mount your google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#training folder
!cp -r "/content/gdrive/MyDrive/Juice_Box/FRCNN/train.zip" "/content/tensorflow-object-detection-faster-rcnn/data"
!unzip '/content/tensorflow-object-detection-faster-rcnn/data/train.zip' -d '/content/tensorflow-object-detection-faster-rcnn/data'

Archive:  /content/tensorflow-object-detection-faster-rcnn/data/train.zip
   creating: /content/tensorflow-object-detection-faster-rcnn/data/round2/
  inflating: /content/tensorflow-object-detection-faster-rcnn/data/round2/clear_plastic_bottle10_jpg.rf.94e003f8976d8ab19a4bf6e9c1357a14.jpg  
  inflating: /content/tensorflow-object-detection-faster-rcnn/data/round2/clear_plastic_bottle10_jpg.rf.94e003f8976d8ab19a4bf6e9c1357a14.xml  
  inflating: /content/tensorflow-object-detection-faster-rcnn/data/round2/clear_plastic_bottle11_jpg.rf.5aea207a1525a5d0bc153f5b85becc2d.jpg  
  inflating: /content/tensorflow-object-detection-faster-rcnn/data/round2/clear_plastic_bottle11_jpg.rf.5aea207a1525a5d0bc153f5b85becc2d.xml  
  inflating: /content/tensorflow-object-detection-faster-rcnn/data/round2/clear_plastic_bottle12_jpg.rf.7e2c63fd733f0c7c921a73dee3e0ef9f.jpg  
  inflating: /content/tensorflow-object-detection-faster-rcnn/data/round2/clear_plastic_bottle12_jpg.rf.7e2c63fd733f0c7c921a73dee3e0ef9f

In [ ]:
#test folder
!cp -r "/content/gdrive/MyDrive/Juice_Box/FRCNN/test" "/content/tensorflow-object-detection-faster-rcnn/data"

In [ ]:
!pip install tensorflow-object-detection-api

     |████████████████████████████████| 583kB 7.9MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 3.2MB 16.1MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-cp37-none-any.whl size=844515 sha256=f39dfe2eb2ffb87cc55596934bdd792334719a9702a4a74e308cbc51df212b63
  Stored in directory: /root/.cache/pip/wheels/4a/54/d0/cfca11930c4b2025d40dede77059094070a67cc3e7bd3b285f
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=1faf47c32f0026af18eb77cf1bfb010f503ff70a5fa404a47a68415926c1ed96
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built tensorflow-object-detection-api gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [ ]:
#Generate csv for train and test labels
!if [ -d '/content/tensorflow-object-detection-faster-rcnn/data/annotations' ]; then rm -r /content/tensorflow-object-detection-faster-rcnn/data/annotations/* ; else mkdir '/content/tensorflow-object-detection-faster-rcnn/data/annotations' && echo "Directory annotations created"; fi

if os.path.exists("/content/tensorflow-object-detection-faster-rcnn/data/annotations/train_labels.csv"):
    os.remove("/content/tensorflow-object-detection-faster-rcnn/data/annotations/train_labels.csv")

if os.path.exists("/content/tensorflow-object-detection-faster-rcnn/data/annotations/test_labels.csv"):
    os.remove("/content/tensorflow-object-detection-faster-rcnn/data/annotations/test_labels.csv")  

rm: cannot remove '/content/tensorflow-object-detection-faster-rcnn/data/annotations/*': No such file or directory


In [ ]:
#Generate csv for train and test labels within annotations folder
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import shutil

annotations = r'/content/tensorflow-object-detection-faster-rcnn/data' 
if not os.path.exists(annotations):
    os.makedirs(annotations)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bndbox = member.find('bndbox')
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member.find('name').text,
                     int(bndbox.find('xmin').text),
                     int(bndbox.find('ymin').text),
                     int(bndbox.find('xmax').text),
                     int(bndbox.find('ymax').text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    xml_df_train = xml_to_csv("/content/tensorflow-object-detection-faster-rcnn/data/train/")
    xml_df_train.to_csv('/content/tensorflow-object-detection-faster-rcnn/data/annotations/'+'train_labels.csv', index=None)
    xml_df_test = xml_to_csv("/content/tensorflow-object-detection-faster-rcnn/data/test/")
    xml_df_test.to_csv('/content/tensorflow-object-detection-faster-rcnn/data/annotations/'+'test_labels.csv', index=None)
    print('Successfully converted xml to csv.')

%cd "/content/tensorflow-object-detection-faster-rcnn"
main()

/content/tensorflow-object-detection-faster-rcnn
Successfully converted xml to csv.


In [ ]:
# Generate tf_label_map.pbtxt in train folder for the classes generated
ssd_classes=pd.DataFrame(pd.read_csv("/content/tensorflow-object-detection-faster-rcnn/data/annotations/train_labels.csv")['class'].unique())
ssd_classes.drop_duplicates(inplace=True)

ssd_classes['id'] = ssd_classes.index

def convert_classes(classes, start=1):
    msg = ''
    for id, name in enumerate(classes, start=start):
        msg = msg + "item {\n"
        msg = msg + " id: " + str(id) + "\n"
        msg = msg + " name: '" + name + "'\n}\n\n"
    return msg[:-1]

label_map = convert_classes(ssd_classes[0])
with open("/content/tensorflow-object-detection-faster-rcnn/data/train/" + "label_map.pbtxt", "w") as f:
    f.write(label_map)
    f.close()    

In [ ]:
# Generate TF record - Run for train
%%capture
%cd {repo_dir_path}
#FLAGS.remove_flag_values(FLAGS.flag_values_dict())

"""
Usage:
# Create train data:
python generate_tfrecord.py --label_map=<PATH_TO_LABEL_MAP_FILE> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/
train_labels.csv --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/train.record

# Create test data:
python generate_tfrecord.py --label_map=<PATH_TO_LABEL_MAP_FILE> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/
test_labels.csv --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
import os
import io
import pandas as pd
import tensorflow as tf
import sys

sys.path.append("../../models/research")
from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.compat.v1.app.flags
flags.DEFINE_string('f','','')
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('img_path', '', 'Path to images')
flags.DEFINE_string('label_map', '', 'Path to label map (.pbtxt) file')

# if your image has more labels input them as
# flags.DEFINE_string('label0', '', 'Name of class[0] label')
# flags.DEFINE_string('label1', '', 'Name of class[1] label')
# and so on.

FLAGS = flags.FLAGS


def class_text_to_int(row_label):
    for label_id, label_name in get_label_info():
        if row_label == label_name:
            return label_id
    return 0


def get_label_info():
    """
    Generate label info from label map (.pbtxt) file
    :return: id, name
    """
    label_info = []
    with open(FLAGS.label_map) as fp:
        for _, line in enumerate(fp):
            if "id:" in line:
                label_id = int(line.split(":")[1])
                label_info.append(label_id)
            elif "name:" in line:
                label_name = line.split(":")[1].strip().replace("'", "")
                label_info.append(label_name)

    for i in range(0, len(label_info), 2):
        yield label_info[i:i + 2]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    filename = group.filename.encode('utf8')
    image_format = b'jpg'

    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    FLAGS.output_path="/content/tensorflow-object-detection-faster-rcnn/data/train/train.tfrecord"
    FLAGS.csv_input="/content/tensorflow-object-detection-faster-rcnn/data/annotations/train_labels.csv"
    FLAGS.img_path="/content/tensorflow-object-detection-faster-rcnn/data/train"
    FLAGS.label_map ="/content/tensorflow-object-detection-faster-rcnn/data/train/label_map.pbtxt"
    
    writer = tf.io.TFRecordWriter("/content/tensorflow-object-detection-faster-rcnn/data/train/train.tfrecord")
    path = os.path.join(os.getcwd(), FLAGS.img_path)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')

    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the Train TFRecord: {}'.format(output_path))


#if __name__ == '__main__':
    #tf.compat.v1.run()
#    with tf.compat.v1.Session() as sess: sess.run()
main(_)

In [ ]:
# Generate TF record - Run for test
%%capture
%cd {repo_dir_path}
FLAGS.remove_flag_values(FLAGS.flag_values_dict())

"""
Usage:
# Create train data:
python generate_tfrecord.py --label_map=<PATH_TO_LABEL_MAP_FILE> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/
train_labels.csv --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/train.record

# Create test data:
python generate_tfrecord.py --label_map=<PATH_TO_LABEL_MAP_FILE> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/
test_labels.csv --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
import os
import io
import pandas as pd
import tensorflow as tf
import sys

sys.path.append("../../models/research")
from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.compat.v1.app.flags
flags.DEFINE_string('f','','')
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('img_path', '', 'Path to images')
flags.DEFINE_string('label_map', '', 'Path to label map (.pbtxt) file')

# if your image has more labels input them as
# flags.DEFINE_string('label0', '', 'Name of class[0] label')
# flags.DEFINE_string('label1', '', 'Name of class[1] label')
# and so on.

FLAGS = flags.FLAGS


def class_text_to_int(row_label):
    for label_id, label_name in get_label_info():
        if row_label == label_name:
            return label_id
    return 0


def get_label_info():
    """
    Generate label info from label map (.pbtxt) file
    :return: id, name
    """
    label_info = []
    with open(FLAGS.label_map) as fp:
        for _, line in enumerate(fp):
            if "id:" in line:
                label_id = int(line.split(":")[1])
                label_info.append(label_id)
            elif "name:" in line:
                label_name = line.split(":")[1].strip().replace("'", "")
                label_info.append(label_name)

    for i in range(0, len(label_info), 2):
        yield label_info[i:i + 2]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    filename = group.filename.encode('utf8')
    image_format = b'jpg'

    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    FLAGS.output_path="/content/tensorflow-object-detection-faster-rcnn/data/test/test.tfrecord"
    FLAGS.csv_input="/content/tensorflow-object-detection-faster-rcnn/data/annotations/test_labels.csv"
    FLAGS.img_path="/content/tensorflow-object-detection-faster-rcnn/data/test"
    FLAGS.label_map ="/content/tensorflow-object-detection-faster-rcnn/data/test/label_map.pbtxt"
    
    writer = tf.io.TFRecordWriter("/content/tensorflow-object-detection-faster-rcnn/data/test/test.tfrecord")
    path = os.path.join(os.getcwd(), FLAGS.img_path)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')

    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the Test TFRecord: {}'.format(output_path))


#if __name__ == '__main__':
    #tf.compat.v1.run()
#    with tf.compat.v1.Session() as sess: sess.run()
main(_)

In [ ]:
# train set
%ls /content/tensorflow-object-detection-faster-rcnn/data/train

clear_plastic_bottle10_jpg.rf.94e003f8976d8ab19a4bf6e9c1357a14.jpg
clear_plastic_bottle10_jpg.rf.94e003f8976d8ab19a4bf6e9c1357a14.xml
clear_plastic_bottle11_jpg.rf.5aea207a1525a5d0bc153f5b85becc2d.jpg
clear_plastic_bottle11_jpg.rf.5aea207a1525a5d0bc153f5b85becc2d.xml
clear_plastic_bottle12_jpg.rf.7e2c63fd733f0c7c921a73dee3e0ef9f.jpg
clear_plastic_bottle12_jpg.rf.7e2c63fd733f0c7c921a73dee3e0ef9f.xml
clear_plastic_bottle13_jpg.rf.963d81fb784379c858201fbf7e7c50d3.jpg
clear_plastic_bottle13_jpg.rf.963d81fb784379c858201fbf7e7c50d3.xml
clear_plastic_bottle14_jpg.rf.24ff1aa17b40c9e3d9591a725fbeb723.jpg
clear_plastic_bottle14_jpg.rf.24ff1aa17b40c9e3d9591a725fbeb723.xml
clear_plastic_bottle15_jpg.rf.e589c91757fe0179107caaa4ed080710.jpg
clear_plastic_bottle15_jpg.rf.e589c91757fe0179107caaa4ed080710.xml
clear_plastic_bottle16_jpg.rf.db349d18ca6cf33a75b03f7d70099603.jpg
clear_plastic_bottle16_jpg.rf.db349d18ca6cf33a75b03f7d70099603.xml
clear_plastic_bottle17_jpg.rf.53877d2c50116a6b67671130071b1854

In [ ]:
test_record_fname = '/content/tensorflow-object-detection-faster-rcnn/data/test/test.tfrecord'
train_record_fname = '/content/tensorflow-object-detection-faster-rcnn/data/train/train.tfrecord'
label_map_pbtxt_fname = '/content/tensorflow-object-detection-faster-rcnn/data/train/label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 111M
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 .
drwxr-xr-x 23 root   root 4.0K Mar 29 21:52 ..
-rw-r--r--  1 345018 5000   77 Feb  1  2018 checkpoint
-rw-r--r--  1 345018 5000  55M Feb  1  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 5000  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 5000  16K Feb  1  2018 model.ckpt.index
-rw-r--r--  1 345018 5000 5.5M Feb  1  2018 model.ckpt.meta
-rw-r--r--  1 345018 5000 3.2K Feb  1  2018 pipeline.config
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        tru

In [ ]:
#model_dir = 'training/'
model_dir = '/content/gdrive/MyDrive/Juice_Box/FRCNN/Checkpoints/'
# Optionally remove content in output model directory to fresh start.
#!rm -rf {model_dir}
#os.makedirs(model_dir, exist_ok=True)

## Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-03-29 21:53:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.159.163.191, 34.196.3.7, 107.21.11.91, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.159.163.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  18.3MB/s    in 0.8s    

2021-03-29 21:53:29 (18.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://34660baf7ba6.ngrok.io


## Train the model

In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps=10000 \
    --num_eval_steps={num_eval_steps}

W0329 23:15:23.140586 140425364469632 model_lib.py:813] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 10000
I0329 23:15:23.140835 140425364469632 config_util.py:552] Maybe overwriting train_steps: 10000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0329 23:15:23.141000 140425364469632 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0329 23:15:23.141162 140425364469632 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0329 23:15:23.141336 140425364469632 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0329 23:15:23.141493 140425364469632 model_lib.py:829] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to

In [ ]:
!ls {model_dir}

checkpoint
eval_0
events.out.tfevents.1617054889.55403818f9f1
events.out.tfevents.1617059775.55403818f9f1
export
graph.pbtxt
model.ckpt-10000.data-00000-of-00001
model.ckpt-10000.index
model.ckpt-10000.meta
model.ckpt-6840.data-00000-of-00001
model.ckpt-6840.index
model.ckpt-6840.meta
model.ckpt-7682.data-00000-of-00001
model.ckpt-7682.index
model.ckpt-7682.meta
model.ckpt-8541.data-00000-of-00001
model.ckpt-8541.index
model.ckpt-8541.meta
model.ckpt-9384.data-00000-of-00001
model.ckpt-9384.index
model.ckpt-9384.meta


In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [ ]:
last_model

'model.ckpt-10000'